In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from sys import maxsize

# Read the data from 'face feature vectors.csv'
data = pd.read_csv('face feature vectors.csv')

# Split the data into train and test sets
train_data = pd.concat([data.iloc[5:400], data.iloc[405:800]])
test_data = pd.concat([data.iloc[0:5], data.iloc[400:405]])

# Calculate class probabilities
p1 = len(train_data.iloc[5:400]) / len(train_data)
p2 = len(train_data.iloc[405:800]) / len(train_data)

# Calculate the dimensions
d = len(train_data.columns) - 2

# Calculate the mean vectors
mean1 = np.mean(train_data.iloc[5:400, 2:130], axis=0).values
mean2 = np.mean(train_data.iloc[405:800, 2:130], axis=0).values

# Calculate the covariance matrices
cov1 = train_data.iloc[5:400, 2:130].cov().values
cov2 = train_data.iloc[405:800, 2:130].cov().values

# Avoid singular matrices
mat = maxsize if np.linalg.det(cov1) == 0 else np.linalg.det(cov1)
mat1 = maxsize if np.linalg.det(cov2) == 0 else np.linalg.det(cov2)

# Define the probability function
def prob(x, mean, cov, p):
    a = 1 / ((2 * math.pi) ** (d / 2))
    b = 1 / (np.power(np.linalg.det(cov), 1 / 2))
    c = (-1 / 2) * (np.dot(np.dot((x - mean).T, np.linalg.inv(cov)), (x - mean)))
    return a * b * math.exp(c) * p

# Classify the test data
results = []
for i in range(len(test_data)):
    x = test_data.iloc[i, 2:130].values
    g1 = prob(x, mean1, cov1, p1)
    g2 = prob(x, mean2, cov2, p2)
    if g1 > g2:
        results.append([test_data.iloc[i, 0], 'male'])
    else:
        results.append([test_data.iloc[i, 0], 'female'])

# Create a DataFrame for the results and save to 'output.csv'
results_df = pd.DataFrame(results, columns=['Id', 'class'])
results_df.to_csv('output.csv', index=False)

# Calculate accuracy
correct_predictions = (results_df['class'] == test_data.iloc[:,1]).sum()
accuracy = correct_predictions / len(test_data) * 100
print("Accuracy is", accuracy)

# Calculate and plot class-wise accuracy
class_counts = test_data.iloc[:,1].value_counts()
class_accuracy = {}
for cls in class_counts.index:
    correct_class_predictions = ((results_df['class'] == cls) & (test_data.iloc[:,1] == cls)).sum()
    class_accuracy[cls] = correct_class_predictions / class_counts[cls]

class_accuracy['overall'] = accuracy

print(accuracy)



C:\Users\kulad\AppData\Local\Temp\ipykernel_31596\2975883903.py:36: RuntimeWarning: divide by zero encountered in double_scalars
  b = 1 / (np.power(np.linalg.det(cov), 1 / 2))


ValueError: Can only compare identically-labeled Series objects